In [43]:
import pandas as pd
import os
import numpy as np
import glob
from os.path import exists


def formatCSV(DataFile):
    """This function adds commas to the first line of a csv to be able to obtain a pandas df"""
    max_commas = 0 # Max number of commmas in single line
    missing_commas = 0 # Missing commas in firts line
    first_commas = 0 # Number of commas in first line
    flag = True
    # Obtain max number of commas in any line in the file
    file = open(DataFile)
    for line in file: 
        if(flag):# Just to get number of commas in the first line
            first_commas = line.count(',')
            flag = False
        if(line.count(',') > max_commas):
            max_commas = line.count(',')
    file.close()

    missing_commas = max_commas- first_commas # Obtain missing commmas if any

    # Add the missing commas to the first line 
    flag = True
    if(missing_commas >0):
        commas = ""
        for ij in range(missing_commas): # Get number of missing commas in string
            commas += ","
        newf=""
        with open(DataFile,'r') as f:
            for line in f:
                if(flag): # Append the commas to just the first line
                    newf += line.strip()+commas+"\n"
                    flag = False
                else:
                    newf += line.strip()+"\n"
                
        with open(DataFile,'w') as f: # Write the newly formed correct file
            f.write(newf)

cell_number = 15 # Number of cells that will be trained
PMFBuffer = [0 for i in range(100)] # Assume signal strength values range from 0 to -100


# Clear the output folder before processing
if (len(os.listdir("model"))!=0):
    files = glob.glob('model/*')
    for f in files:
        os.remove(f)


# Format the input data to represent proper csv
for i in range(1):
    file_name = "TrainingData/trainMonday16/saved_data_cell" + str(i+1)+ "_monday_all.txt"
    # Format the training data
    with open(file_name, 'r') as my_file:
        text = my_file.read()
        text = text.replace("[", "")
        text = text.replace("]", "")
        text = text.replace(" ", "")
    with open(file_name, 'w') as my_file:
        my_file.write(text)


# Iterate over files for each cells training data
for i in range(cell_number):
    trainingDataFile = "TrainingData/trainMonday16/saved_data_cell" + str(i+1)+ "_monday_all.txt"

    # Format each file before processing
    formatCSV(trainingDataFile)

    df_init = pd.read_csv(trainingDataFile) # Read the cell data into a df
    sampleSize = df_init.shape[1]-1 # Maximum number of samples for a single measurement

    # Iterate over each row of the dataframe
    for index, row in df_init.iterrows():
        file_name_MAC = "model/" + row[0] + ".csv" # File where the radar map will be placed
        
        if(exists(file_name_MAC)): # If the file already exists insert new row

            for j in range(sampleSize): # Obtain the histogram by increasing the array by index of the present signal
                if(isinstance(row[j+1], int)): # Check if there is a sample at the index. not NaN
                    PMFBuffer[int(-row[j+1])] +=1

            # Obtain sum of the histogram for normalization
            sum = 0;     
            for j in range(0, len(PMFBuffer)):    
                sum = sum + PMFBuffer[j];  

            for j in range(len(PMFBuffer)): # Normalize histogram for pmf
                PMFBuffer[j] = PMFBuffer[j]/float(sum)

            df_tmp1 = {'Cells': "cell"+str(i+1), 'PMF':PMFBuffer} # Temp df    
            
            df_tmp = pd.read_csv(file_name_MAC) # Read the map to insert new pmf
            df_tmp = df_tmp.append(df_tmp1, ignore_index = True)
            df_tmp.to_csv(file_name_MAC, index=False)
            PMFBuffer = [0 for q in range(100)]

        else: # If the file does not exist, create and format the file

            for j in range(sampleSize): # Obtain the histogram by increasing the array by index of the present signal
                if(isinstance(row[j+1], int)): # Check if there is a sample at the index. not NaN
                    PMFBuffer[int(-row[j+1])] +=1
            
            # Obtain sum of the histogram for normalization
            sum = 0;     
            for j in range(0, len(PMFBuffer)):    
                sum = sum + PMFBuffer[j]; 

            for j in range(len(PMFBuffer)): # Normalize histogram for pmf
                PMFBuffer[j] = PMFBuffer[j]/float(sum)

            df_tmp = pd.DataFrame([['cell' + str(i+1), PMFBuffer]])
            df_tmp.columns = ['Cells', 'PMF']
            df_tmp.to_csv(file_name_MAC, index=False)
            PMFBuffer = [0 for q in range(100)]


